 ## Part 1: Entanglement Swapping Protocol 
    
As opposed to teleportation of a single quantum state, entanglement ewapping involves the transfer (teleportation) of entanglement to two qubits that were produced independently and have never previously interacted. It is used to extend the range of shared entanglement.

**Goal:** The purpose is to demonstrate how entanglement swapping is performed between two parties, where two bell pairs (maximally entangled state) generated locally are converted to a single bell pair shared between the users (pg 150 of The Quantum Internet, Rhode, P). The steps are as follows:

- Alice and Bob generate EPR pairs $|\Phi^+\rangle_{AB}$ and $|\Phi^+\rangle_{CD}$ seperateley, resulting in the composite state $|\Psi\rangle_{ABCD} = |\Phi^+\rangle_{AB}|\Phi^+\rangle_{CD} $. 
- Each transmits half of their pair (qubit B and C) to Carol, who performs a bell projection between B and C yielding by chance $\langle\Phi^+\rangle_{AB} |\Psi\rangle_{ABCD} = |\Psi^+\rangle_{AD}$ (or $|\Phi^-\rangle_{AD}$, $|\Psi^-\rangle_{AD}$, or $|\Phi^+\rangle_{AD}$). 
- Carol then sends the measurement results to Bob, who performs unitary pauli transformations necessary to obtain the $|\Phi^+\rangle_{AD}$ state in the case of the other 3 bell projection outcomes.

Circuit diagram shown below, from  Michael R. Grimaila's Presentation \Understanding Superdense Coding, Quantum Teleportation, and Entanglement Swapping Protocols\ (2023)

<!--> <img src=\img/entanglement_swap.png\ width=\800\>\n <-->

In [2]:
from squanch import *
import numpy as np
import matplotlib.image as image
import matplotlib.pyplot as plt
import multiprocessing

multiprocessing.set_start_method("fork", force=True)


In [4]:
class Alice(Agent):
    '''Alice wishes to entangle her qubit with Bob's"'''
    def run(self):
        measurements = []
        for qsys in self.qstream:
            a, b, _, _ = qsys.qubits

            # Locally prepare bell pair |Φ⁺⟩_AB
            H(a)
            CNOT(a, b)

            # Send qubit b to Carol
            self.qsend(carol, b)

            # (To verify)  Measure qubit a after Bob measures his
            m_d = self.crecv(bob)
            measurements.append([a.measure(), m_d])
            
        self.output(measurements)

In [5]:
class Carol(Agent):
    """ Carol performs bell projection to link Alice and Bob's qubits"""
    def run(self):
        for _ in self.qstream:
            # Receive qubits b and c from Alice and Bob
            b = self.qrecv(alice)
            c = self.qrecv(bob)

            # Perform bell state measurment between b and c, entangling qubit a and d
            CNOT(b, c)
            H(b)
            b1, b2 = bool(b.measure()), bool(c.measure())

            # Forward measurments to Bob
            self.csend(bob, [b2, b1])

        self.output("Carol done")

In [6]:
class Bob(Agent):
    """ Bob wishes to entangle his qubit with Alice's"""
    def run(self):
        for qsys in self.qstream:
            _, _, c, d = qsys.qubits

            # Locally prepare bell pair |Φ⁺⟩_CD
            H(c)
            CNOT(c, d)

            # Send qubit c to Carol
            self.qsend(carol, c)

            # Receive bell state measurement from Carol, apply unitary trasformation to convert to |Φ⁺⟩_ad
            should_apply_x, should_apply_z = self.crecv(carol)
            if should_apply_x:
                X(d)
            if should_apply_z:
                Z(d)

            # Send measure to ALice
            self.csend(alice, d.measure())

        self.output("Bob done")

In [7]:
# Initialize quantum stream
qstream = QStream(4, 10)

# Setup Agent instances
out = Agent.shared_output()
alice = Alice(qstream, out = out)
bob = Bob(qstream, out = out)
carol = Carol(qstream, out = out)

# Connect Agents with quantum and classical channels
alice.qconnect(carol) # add a quantum channel
bob.qconnect(carol)
carol.cconnect(bob)
bob.cconnect(alice) # add a classical channel

In [8]:
Simulation(alice, bob, carol).run(monitor_progress=False)
print("Alice,Eve Measures: {}\n".format(out["Alice"]))

Alice,Eve Measures: [[0, 0], [0, 0], [1, 1], [1, 1], [1, 1], [1, 1], [0, 0], [1, 1], [1, 1], [1, 1]]



 ## Part 1: Quantum Repeater Chain (2 intermediate nodes)

In [9]:
class Alice(Agent):
    '''Alice wishes to entangle her qubit with Bob's"'''
    def run(self):
        measurements = []
        for qsys in self.qstream:
            a, b, _, _, _, _= qsys.qubits

            # Locally prepare bell pair |Φ⁺⟩_AB
            H(a)
            CNOT(a, b)

            # Send qubit b to Carol
            self.qsend(carol, b)

            # (To verify)  Measure qubit a after Bob measures his
            m_f = self.crecv(eve)
            measurements.append([a.measure(), m_f])
            
        self.output(measurements)

In [10]:
class Bob(Agent):
    """ Bob wishes to entangle his qubit with Alice's"""
    def run(self):
        for qsys in self.qstream:
            _, _, c, d, _, _= qsys.qubits

            # Locally prepare bell pair |Φ⁺⟩_CD
            H(c)
            CNOT(c, d)

            # Send qubit c to Carol
            self.qsend(carol, c)


            # Receive bell state measurement from Carol, apply unitary trasformation to convert to |Φ⁺⟩_ad
            should_apply_x, should_apply_z = self.crecv(carol)
            if should_apply_x:
                X(d)
            if should_apply_z:
                Z(d)
            
            # Send qubit d (now entangled w/ Alice's qubit a) to Doug
            self.qsend(doug, d)

        self.output("Bob done")

In [11]:
class Carol(Agent):
    """ Carol performs bell projection to link Alice and Bob's qubits"""
    def run(self):
        for _ in self.qstream:
            # Receive qubits b and c from Alice and Bob
            b = self.qrecv(alice)
            c = self.qrecv(bob)

            # Perform bell state measurment between b and c, entangling qubit a and d
            CNOT(b, c)
            H(b)
            b1, b2 = bool(b.measure()), bool(c.measure())

            # Forward measurments to Bob
            self.csend(bob, [b2, b1])

        self.output("Carol done")

In [12]:
class Eve(Agent):
    """ Eve wishes to entangle her qubit with Alice's"""
    def run(self):
        for qsys in self.qstream:
            _, _, _, _, e, f = qsys.qubits

            # Locally prepare bell pair |Φ⁺⟩_EF
            H(e)
            CNOT(e, f)

            # Send qubit e to Doug
            self.qsend(doug, e)

            # Receive bell state measurement from Doug, apply unitary trasformation to convert to |Φ⁺⟩_af
            should_apply_x, should_apply_z = self.crecv(doug)
            if should_apply_x:
                X(f)
            if should_apply_z:
                Z(f)

            # (To verify)  Measure qubit f, notify Alice to measure her half
            m = f.measure()
            self.csend(alice, m)
    
        self.output("Eve done")

In [13]:
class Doug(Agent):
    """ Doug performs bell projection to link Alice and Eve's qubits"""
    def run(self):
        for _ in self.qstream:
            # Receive qubits b and c from Alice and Bob
            d = self.qrecv(bob)
            e = self.qrecv(eve)

            # Perform bell state measurment between b and c, entangling qubit a and d
            CNOT(d, e)
            H(d)
            b1, b2 = bool(d.measure()), bool(e.measure())

            # Forward measurments to Eve
            self.csend(eve, [b2, b1])

        self.output("doug done")

In [14]:
# Initialize quantum stream
qstream = QStream(6, 10)

# Setup Agent instances
out = Agent.shared_output()
alice = Alice(qstream, out = out)
bob = Bob(qstream, out = out)
carol = Carol(qstream, out = out)
doug = Doug(qstream, out = out)
eve = Eve(qstream, out = out)

# Connect Agents with quantum and classical channels
alice.qconnect(carol) # add a quantum channel
bob.qconnect(carol)
carol.cconnect(bob)
bob.qconnect(doug)
eve.qconnect(doug) 
doug.cconnect(eve) 
eve.cconnect(alice) # add a classical channel

In [15]:
Simulation(alice, bob, carol, doug, eve).run(monitor_progress=False)
print("Alice,Eve Measures: {}\n".format(out["Alice"]))

Alice,Eve Measures: [[1, 1], [0, 0], [1, 1], [0, 0], [1, 1], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0]]

